Spatial resolution measures how much real world area will get mapped into a single pixel of the satellite raster image. For example, if the resolution is 20 meters, then a 20 by 20 meter area will get reflected as only a single pixel. With such a resolution it is not possible to identify individual cars, and even buildings can be identified only if they are sufficiently large.

Different satellite images come with different spatial resolution. In this notebook we want to compare some example datasets in order to get a better feeling about spatial resolutions.

In [ ]:
import geemap
import ee
from egis.utils import (
    vis_params_dict, extract_single_img
)

In [ ]:
ee.Initialize()

### Select example area

First, we define a region of interest (ROI) for our comparison. We pick the southern part of Manhattan in this example.

In [ ]:
coord_list = [[-74.021956, 40.699491],
              [-73.998517, 40.699491],
              [-73.998517, 40.714391],
              [-74.021956, 40.714391],
              [-74.021956, 40.699491]]
roi_clip_region = ee.Geometry.Polygon(coord_list)

# compute center point
roi_center_point = roi_clip_region.centroid()

# extract coordinates for centering of maps
roi_center_coords = roi_center_point.getInfo()['coordinates']
roi_center_coords = [roi_center_coords[1], roi_center_coords[0]]

### Google static map

The first dataset that we inspect are the satellite images available from Google maps. This is actually a pretty awesome dataset, because it has a lot of complicated cleaning and processing steps in the background, as it is a composition of multiple smaller images:

- from several data providers
- from several points in time
- with different resolutions

Overall, we get one large image that has very high resolution in many parts of the world (big cities). And it actually has another convenient property: the large image is composed in such a way that almost all regions of the world are visible without any clouds.

In [ ]:
google_map = geemap.Map(center=roi_center_coords, zoom=16)
google_map.add_basemap('SATELLITE')
google_map

### Sentinel 2

Our second example is an image of the Sentinel 2 mission. In comparison to the Google static map, there are multiple images for each point of interest from multiple periods of time. This comes with the advantage that one potentially could both select the region of interest and the point in time of the image. In this example I picked an arbitrary historic image without clouds.

In [ ]:
sentinel_image = ee.Image('COPERNICUS/S2_SR_HARMONIZED/20221027T155341_20221027T155837_T18TWL')

In [ ]:
vis_params = vis_params_dict(bands=['B4','B3','B2'],
                                     min=0,
                                     max=3000,
                                     gamma=1.4)

In [ ]:
sentinel_map = geemap.Map()
sentinel_map.addLayer(sentinel_image, vis_params)
sentinel_map.centerObject(roi_center_point, 16)
sentinel_map

As we can see, the spatial resolution of the Sentinel 2 image is significantly worse than what we did get from the Google static map image.

### National Agriculture Imagery Program

As a last example we pick an image from the [National Agriculture Imagery Program](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ) in the US. This dataset also comes with a very good resolution and even snapshots over time. However, it does not span over the full globe.

In [ ]:
naip_img = ee.Image('USDA/NAIP/DOQQ/m_4007424_ne_18_1_20170719')

In [ ]:
vis_params_naip = vis_params_dict(bands=['R','G','B'], min=0.0, max=255.0, gamma=0.6) 

In [ ]:
naip_map = geemap.Map()
naip_map.addLayer(naip_img, vis_params_naip)
naip_map.centerObject(roi_center_point, 16)
naip_map